In [2]:
import os
from langchain.document_loaders import PagedPDFSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.document_loaders import YoutubeLoader
loaders = []



def get_pdf_files(filepath):
    for filename in os.listdir(filepath):
        if filename.endswith('.pdf'):
            yield filepath + filename


## replace with pdf file paths here
path = 'pdfs/'
for filename in get_pdf_files(path):
    loaders.append(PagedPDFSplitter.from_pdf(filename))


## replace with any youtube videos here
youtube_files = [
    'https://www.youtube.com/watch?v=_hWz0V3Kq64',
    'https://www.youtube.com/watch?v=-UO0vKojTn4',
    'https://www.youtube.com/watch?v=6DT6y49qV9I',
    'https://www.youtube.com/watch?v=oa0hcI4kWqU',
    'https://www.youtube.com/watch?v=ljm_wuMOTQM',
    'https://www.youtube.com/watch?v=ce0n6WrXREM',
]

for filename in youtube_files:
    loaders.append(YoutubeLoader.from_youtube_url(filename))

docsearch = VectorstoreIndexCreator().from_loaders(loaders)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [3]:
from langchain import OpenAI, VectorDBQA
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    ## make this your own key
    api_key=os.environ['OPENAI_API_KEY'],
    ## change this
    model='gpt-4'
)

notes = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=docsearch.vectorstore)


In [96]:
from langchain.agents import initialize_agent, Tool

tools = [
    Tool(
        name="Notes QA System",
        description="useful for when you need context from the notes to provide an answer to a given question. Input should be a fully formed question.",
        func=notes.run
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, max_iterations=5, agent_kwargs={
    'prefix':"""You will be answering as best as possible multiple choice questions about the course material. 
    If you don't know the answer, and cannot find the answer with the help og the notes, you should say "I don't know".
    
    You have access to the following tools:
    """,
    "input_variables":[
    "question",
    "answers",
    "agent_scratchpad"
    ],
    'suffix':"""

    Begin! 
    Remember, The only action you can use is `Notes QA System`.
    Here is the question you need to answer:
    {question}
    Here are the possible answers:
    {answers}

    {agent_scratchpad}
    """
})


In [99]:
agent.run(
    question="""
When there is a possibility of an extinction burst, you should determine whether:​
    """,
    answers="""
a.
punishment can be implemented for novel behaviors
b.
the extinction burst is desirable
c.
the change agent can withhold the reinforcer
d.
A & C
    """
)




> Entering new AgentExecutor chain...

Question: When there is a possibility of an extinction burst, you should determine whether:
Thought: I need to look at the notes to answer this question
Action: Notes QA System
Action Input: What should I do when there is a possibility of an extinction burst?
Observation:  When there is a possibility of an extinction burst, you should make sure that you have identified the reinforcer(s) and that you can control them. You should also make sure that it is possible to eliminate the reinforcer(s) and that it is safe to use extinction. Additionally, you should ensure that you can apply extinction consistently and that you have procedures in place to promote generalization and maintenance.
Thought: I now know the answer to the original question
Final Answer: d. A & C

> Finished chain.


'd. A & C'

**Anything below this cell does not use an agent, and may provide different results.**

In [29]:
multiple_choice_prompt_template = """
Answer the following mutliple choice question by selecting the correct answer from the options below.
{question}
Use the context to help support your answer.
{context}

Output the answer as either A, B, C, or D along with the justification for your answer.
You MUST include the justification for your answer.

If the context does not provide enough information to answer the question, you can say "I don't know" and the question will be skipped.
"""

In [30]:
from langchain import PromptTemplate
prompt = PromptTemplate(template=multiple_choice_prompt_template, 
                        input_variables=['context', 'question']
                        )

In [7]:
prompt_parse_template = """
Parse the following input into this JSON format: 
    \"answer_a\": \"A\",
    \"answer_b\": \"B\",
    \"answer_c\": \"C\",
    \"answer_d\": \"D\",

Ignore when is says `cross out`. That is just a placeholder for the answer. 
Be sure to wrap the answer in brackets.
Here's the input:
{input}
"""

parse_prompt = PromptTemplate(template=prompt_parse_template, input_variables=['input'])
from langchain import LLMChain

llm = OpenAI(temperature=0.0)

parse_chain = LLMChain(llm=llm, prompt=parse_prompt)




In [92]:
from langchain.chat_models import ChatOpenAI

## Change query to your own question
query = """
Johnny hit his sister while they were playing and as a result, his mother made him sit in a chair in the guest bedroom by himself for 5 minutes. What procedure is illustrated in this example?

a.
exclusionary time-out


b.
nonexclusionary time-out


c.
response cost


d.
extinction
"""
question_docs = docsearch.vectorstore.similarity_search(query)


llm = ChatOpenAI(temperature=0.0, model_name="gpt-4")
chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)


print(chain.run({
    "input_documents": question_docs,
    "question": query
}))

A. exclusionary time-out

The procedure illustrated in this example is an exclusionary time-out because Johnny is completely removed from the situation and taken to another room (the guest bedroom) as a consequence of his behavior. This is different from a nonexclusionary time-out, where the individual remains in the same room but loses access to positive reinforcement.
